# Calculate several contrasts using no sex model

In [1]:
setwd('~/5XFAD_mouse/data/')

In [2]:
suppressPackageStartupMessages({
    library(Seurat)
    library(SeuratWrappers)
    library(patchwork)
    library(ggplot2)
    library(repr)
    library(gridExtra)
    library(edgeR)
    library(SingleCellExperiment)
    library(Matrix)
    library(scran)
    library(tidyverse)
    library(ggrepel)
    library(scater)
})
options(future.globals.maxSize = 1e9)
options(Seurat.object.assay.version = "v5")
options(ggrepel.max.overlaps = Inf)

In [3]:
curr.model <- readRDS("DAR_fit_no_sex_models/Astro.model")

In [4]:
contrasts.list <- makeContrasts(
    SedvsEx.9M.WT=WT.9M.Sed-WT.9M.Ex,
    SedvsEx.9M.AD=AD.9M.Sed-AD.9M.Ex,
    SedvsEx.18M.WT=WT.18M.Sed-WT.18M.Ex,
    SedvsEx.18M.AD=AD.18M.Sed-AD.18M.Ex,
    levels = curr.model$design
)

In [5]:
contrast.names <- c(
    "SedvsEx.9M.WT",
    "SedvsEx.9M.AD",
    "SedvsEx.18M.WT",
    "SedvsEx.18M.AD"
)

In [6]:
model.names <- list.files("DAR_fit_no_sex_models/")
celltype.list <- sapply(strsplit(model.names, split = "[.]"), `[`, 1)
celltype.list

[1] "Astro"          "CA1"            "CA2"            "CA3"           
 [5] "Cajal-Retzius"  "Chandelier"     "Choroid-plexus" "DG"            
 [9] "Endo"           "IOL"            "Lamp5"          "Meis2"         
[13] "Microglia"      "NB"             "Oligo"          "OPC"           
[17] "Pvalb"          "PVM"            "RGL"            "SMC-Peri"      
[21] "Sncg"           "Sst"            "SUB_1"          "SUB_2"         
[25] "SUB_3"          "SUB-ProS"       "Vip"            "VLMC"

In [7]:
for (celltype in celltype.list) {
    curr.model <- readRDS(file.path("DAR_fit_no_sex_models/", paste0(celltype, ".model")))
    curr.output.dir <- file.path("DAR_contrasts_no_sex/", celltype)
    if (!dir.exists(curr.output.dir)) {
        dir.create(curr.output.dir)
    }
    
    for (contrast in contrast.names) {
        curr.glf <- glmQLFTest(curr.model, contrast = contrasts.list[, contrast])
        write.table(
            topTags(curr.glf, n = Inf)$table, 
            file.path(curr.output.dir, paste0(contrast, ".", "tsv")), 
            row.names = TRUE,
            col.names = TRUE,
            quote = FALSE,
            sep = "\t"
        )
    }
}